# preprocess revise

- 항 내용 속에 "제n조n항"이 존재하는 경우, 이상하게 잘리는 경우 존재 (index: null인 경우로 검색하여 확인 가능)

- 이를 해결하기 위한 ipynb 파일/

In [ ]:
# 항 이상하게 잘린 것들 고치기 - law_main_supplement

import json
from tqdm import tqdm
import copy

with open("../results/1-2/DCM_1-2_law_supplementary.json", "r", encoding="utf-8") as file:
    data = json.load(file)

len(data)

In [ ]:
prev_article = 0

remove_idx = []
exception_idx = []

for idx, row in tqdm(enumerate(data)):
    if row['subtitle'] != None:
        prev_article = idx
    else:
        # 이전 row에 덧붙이고 현재 인덱스는 삭제
        if type(row['content']) == str:
            # 이전 row가 list 형태일 경우
            if type(data[prev_article]['content']) == list:
                data[prev_article]['content'][-1] += "\n" + row['index'] + row['content']
            # 이전 row가 단일 항 (str) 형태일 경우
            else:
                data[prev_article]['content'] += "\n" + row['index'] + row['content']

        else: # 항 분리
            # 이전 row가 list 형태일 경우
            if type(data[prev_article]['content']) == list:
                data[prev_article]['content'][-1] += "\n" + row['index'] + row['content'][0]
                data[prev_article]['content'] += row['content'][1:]
            
            # 이전 row가 단일 항 (str) 형태일 경우
            else:
                data[prev_article]['content'] += "\n" + row['index'] + row['content'][0]
                data[prev_article]['content'] = [data[prev_article]['content']] + row['content'][1:]
                print(data[prev_article]['content'])

        remove_idx.append(idx)

In [ ]:
print(f"삭제될 항 개수 : {len(remove_idx)}")

In [ ]:
def remove_elements_by_indices(original_list, indices_to_remove):
    # 인덱스 번호 리스트를 내림차순으로 정렬
    indices_to_remove = sorted(indices_to_remove, reverse=True)
    
    # 각 인덱스에 해당하는 요소를 리스트에서 제거
    for index in indices_to_remove:
        if 0 <= index < len(original_list):  # 인덱스가 유효한 경우에만 제거
            del original_list[index]
    
    return original_list

new_data = remove_elements_by_indices(data, remove_idx)

In [ ]:
print(f"보존될 항 개수 : {len(new_data)}")

In [ ]:
import os

path = "../results/1-2/DCM_1-2_law_supplementary_revised.json"
path_list = path.split('/')

os.makedirs(os.path.join("../results", path_list[2]), exist_ok=True)
RESULT_PATH = os.path.join("../results", path_list[2], path_list[3].split('.')[0] + ".json")

print(RESULT_PATH)

with open(RESULT_PATH, "w", encoding="utf-8") as f:
    json.dump(data, f, ensure_ascii=False, indent=4)